In [1]:
from openclean.data.source.socrata import Socrata

for dataset in Socrata().catalog(domain='data.cityofnewyork.us'):
    if 'citywide' in dataset.name.lower() and 'payroll' in dataset.name.lower():
        print(f'{dataset.identifier}\t{dataset.domain}\t{dataset.name}')

k397-673e	data.cityofnewyork.us	Citywide Payroll Data (Fiscal Year)


In [2]:
import gzip
import humanfriendly
import os

dataset = Socrata().dataset('k397-673e')
datafile = './k397-673e.tsv.gz'

if not os.path.isfile(datafile):
    with gzip.open(datafile, 'wb') as f:
        print('Downloading ...\n')
        dataset.write(f)
        
fsize = humanfriendly.format_size(os.stat(datafile).st_size)
print("Using '{}' in file {} of size {}".format(dataset.name, datafile, fsize))


Using 'Citywide Payroll Data (Fiscal Year)' in file ./k397-673e.tsv.gz of size 89.62 MB


In [3]:
from openclean.pipeline import stream

ds_full = stream(datafile)

In [4]:
print(f'{ds_full.count():,} rows.')

3,923,290 rows.


In [5]:
ds_full.head()

,Fiscal Year,Payroll Number,Agency Name,Last Name,First Name,Mid Init,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay
0,2020,17,OFFICE OF EMERGENCY MANAGEMENT,BEREZIN,MIKHAIL,,08/10/2015,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,per Annum,1820,84698.21,0,0.00,0.00
1,2020,17,OFFICE OF EMERGENCY MANAGEMENT,GEAGER,VERONICA,M,09/12/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,per Annum,1820,84698.21,0,0.00,0.00
2,2020,17,OFFICE OF EMERGENCY MANAGEMENT,RAMANI,SHRADDHA,,02/22/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,per Annum,1820,84698.21,0,0.00,0.00
3,2020,17,OFFICE OF EMERGENCY MANAGEMENT,ROTTA,JONATHAN,D,09/16/2013,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,per Annum,1820,84698.21,0,0.00,0.00
4,2020,17,OFFICE OF EMERGENCY MANAGEMENT,WILSON II,ROBERT,P,04/30/2018,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,per Annum,1820,84698.21,0,0.00,0.00
5,2020,17,OFFICE OF EMERGENCY MANAGEMENT,WASHINGTON,MORIAH,A,03/18/2019,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,per Annum,1820,87900.95,0,0.00,-3202.74
6,2020,17,OFFICE OF EMERGENCY MANAGEMENT,VAZQUEZ,MARGARET,,09/29/2008,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,94415.00,per Annum,1820,84312.72,0,0.00,0.00
7,2020,17,OFFICE OF EMERGENCY MANAGEMENT,KRAWCZYK,AMANDA,N,05/15/2017,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,per Annum,1820,83976.54,0,0.00,0.00
8,2020,17,OFFICE OF EMERGENCY MANAGEMENT,MURRELL,JALEESA,S,12/01/2014,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,per Annum,1820,83877.36,0,0.00,0.00
9,2020,17,OFFICE OF EMERGENCY MANAGEMENT,DE LOS SANTOS,JANIRA,,06/05/2017,BROOKLYN,EMERGENCY PREPAREDNESS SPECIALIST,ACTIVE,67676.00,per Annum,1820,66647.77,348.5,16572.64,144.15


In [6]:
COLUMNS = [
    "Fiscal Year",
    "Payroll Number",
    "Agency Name",
    "Last Name",
    "First Name",
    "Mid Init",
    "Agency Start Date",
    "Work Location Borough",
    "Title Description",
    "Leave Status as of June 30",
    "Base Salary",
    "Pay Basis",
    "Regular Hours",
    "Regular Gross Paid",
    "OT Hours",
    "Total OT Paid",
    "Total Other Pay"
]

ds = ds_full.select(columns=COLUMNS)

It was noticed that there were empty values for payroll number, names, work location borough and title description, which should be fixed during data cleaning procedure

In [19]:
from openclean.profiling.column import DefaultColumnProfiler

profiles = ds.profile(default_profiler=DefaultColumnProfiler)

In [14]:
profiles.stats()

,total,empty,distinct,uniqueness,entropy
Fiscal Year,3923290,0,7,0.000002,2.805614
Payroll Number,3923290,1745440,157,0.000072,4.286506
Agency Name,3923290,0,165,0.000042,4.365925
Last Name,3923290,2031,157080,0.040059,14.264455
First Name,3923290,2033,88232,0.022501,11.611521
Mid Init,3923290,1596166,43,0.000018,4.073274
Agency Start Date,3923290,63,14933,0.003806,11.097847
Work Location Borough,3923290,506226,22,0.000006,1.507244
Title Description,3923290,84,1802,0.000459,6.207524
Leave Status as of June 30,3923290,0,5,0.000001,0.710495


In [9]:
print('Schema\n------')
for col in ds.columns:
    p = profiles.column(col)
    print("  '{}' ({})".format(col, p['datatypes']['distinct'].most_common(1)[0][0]))

Schema
------
  'Fiscal Year' (int)
  'Payroll Number' (int)
  'Agency Name' (str)
  'Last Name' (str)
  'First Name' (str)
  'Mid Init' (str)
  'Agency Start Date' (date)
  'Work Location Borough' (str)
  'Title Description' (str)
  'Leave Status as of June 30' (str)
  'Base Salary' (float)
  'Pay Basis' (str)
  'Regular Hours' (float)
  'Regular Gross Paid' (float)
  'OT Hours' (float)
  'Total OT Paid' (float)
  'Total Other Pay' (float)


Total 506,226 rows of work location borough were empty, and 83688 rows were with OTHER value

In our strategy, during data cleaning, we filled the work location borough with empty value by OTHER

And we also implemented data reference to check the correctness of the borough name, if the borough does not exist, it should be other

In [25]:
work_loc = ds.select('Work Location Borough').distinct()
for rank, val in enumerate(work_loc.most_common()):
    st, freq = val
    print(f'{rank + 1:<3} {st}  {freq:>10,}')

1   MANHATTAN   2,394,979
2        506,226
3   QUEENS     379,695
4   BROOKLYN     323,565
5   BRONX     177,881
6   OTHER      83,688
7   RICHMOND      46,156
8   WESTCHESTER       3,417
9   ULSTER       1,953
10  Manhattan       1,622
11  Bronx         935
12  SULLIVAN         822
13  Queens         660
14  DELAWARE         551
15  NASSAU         245
16  PUTNAM         243
17  SCHOHARIE         175
18  DUTCHESS         140
19  Richmond         112
20  ALBANY          95
21  GREENE          61
22  WASHINGTON DC          47
23  ORANGE          22


In [35]:
from openclean.function.value.null import is_empty

# It was found that in original dataset, the 'Work Location Borough' col has empty values
# In our strategy, we merged these columns to OTHER category
# It was also noticed that there were some values with title case, we unified them with upper case

ds = ds.update('Work Location Borough', str.upper).update('Work Location Borough', lambda x: 'OTHER' if is_empty(x) else x)
work_loc = ds.select('Work Location Borough').distinct()

print("Total {0} work locations".format(len(work_loc)))
for rank, val in enumerate(work_loc.most_common()):
    st, freq = val
    print(f'{rank + 1:<3} {st}  {freq:>10,}')

Total 18 work locations
1   MANHATTAN   2,396,601
2   OTHER     589,914
3   QUEENS     380,355
4   BROOKLYN     323,565
5   BRONX     178,816
6   RICHMOND      46,268
7   WESTCHESTER       3,417
8   ULSTER       1,953
9   SULLIVAN         822
10  DELAWARE         551
11  NASSAU         245
12  PUTNAM         243
13  SCHOHARIE         175
14  DUTCHESS         140
15  ALBANY          95
16  GREENE          61
17  WASHINGTON DC          47
18  ORANGE          22


In [12]:
titles = ds.distinct('Title Description')
for rank, val in enumerate(titles.most_common()):
    st, freq = val
    print(f'{rank + 1:<3} {st}  {freq:>10,}')

1   TEACHER- PER SESSION     594,024
2   TEACHER     435,112
3   ELECTION WORKER     232,159
4   ANNUAL ED PARA     191,944
5   TEACHER SPECIAL EDUCATION     181,519
6   POLICE OFFICER     180,668
7   TEACHER-GENERAL ED      84,260
8   CORRECTION OFFICER      69,696
9   FIREFIGHTER      62,057
10  F/T SCHOOL AIDE      58,867
11  JOB TRAINING PARTICIPANT      55,701
12  SUBSTITUTE ED PARA      51,345
13  SANITATION WORKER      47,151
14  COMMUNITY ASSOCIATE      40,638
15  SCHOOL SAFETY AGENT      39,776
16  COLLEGE ASSISTANT      37,817
17  ADJUNCT LECTURER      30,080
18  CLERICAL ASSOCIATE      29,682
19  F/T SCHOOL LUNCH HELPER      29,590
20  CITY SEASONAL AIDE      27,562
21  ASSISTANT PRINCIPAL      26,839
22  CARETAKER      26,365
23  COMMUNITY COORDINATOR      26,326
24  GUIDANCE COUNSELOR      24,407
25  SCHOOL SECRETARY      24,034
26  P.O. DA DET GR3      23,949
27  STUDENT AIDE      22,893
28  LIEUTENANT      22,863
29  EMERGENCY MEDICAL SPECIALIST-EMT      20,038
30  F/T S

In [39]:
# Same as column 'Work Location Borough', according to the stats, and also some titles were not in upper case
# there are also 84 rows in 'Title Description', we merged it with value 'UNKNOWN' and update the value with upper case
# It was also noticed that for some titles, there were special characters '*' and '?' at the beginning
# We cleaned these characters with strip function

import re

ds = ds.update('Title Description', str.upper)\
.update('Title Description', lambda x: 'UNKNOWN' if is_empty(x) else x)\
.update('Title Description', lambda x: re.sub("^[*\?]*", "", str(x)))
titles = ds.distinct('Title Description')
print("Total {0} titles".format(len(titles)))
for rank, val in enumerate(titles.most_common()):
    st, freq = val
    print(f'{rank + 1:<3} {st}  {freq:>10,}')

Total 1762 titles
1   TEACHER- PER SESSION     594,024
2   TEACHER     435,112
3   ELECTION WORKER     232,159
4   ANNUAL ED PARA     191,944
5   TEACHER SPECIAL EDUCATION     181,519
6   POLICE OFFICER     180,668
7   TEACHER-GENERAL ED      84,260
8   CORRECTION OFFICER      69,696
9   FIREFIGHTER      62,057
10  F/T SCHOOL AIDE      58,867
11  JOB TRAINING PARTICIPANT      55,701
12  SUBSTITUTE ED PARA      51,345
13  SANITATION WORKER      47,151
14  COMMUNITY ASSOCIATE      40,638
15  SCHOOL SAFETY AGENT      39,776
16  COLLEGE ASSISTANT      37,817
17  ADJUNCT LECTURER      30,080
18  CLERICAL ASSOCIATE      29,682
19  F/T SCHOOL LUNCH HELPER      29,590
20  CITY SEASONAL AIDE      27,562
21  ASSISTANT PRINCIPAL      26,839
22  CARETAKER      26,365
23  COMMUNITY COORDINATOR      26,326
24  GUIDANCE COUNSELOR      24,407
25  SCHOOL SECRETARY      24,034
26  P.O. DA DET GR3      23,949
27  STUDENT AIDE      22,893
28  LIEUTENANT      22,863
29  EMERGENCY MEDICAL SPECIALIST-EMT    

In [13]:
states = ds.distinct('Leave Status as of June 30')
for rank, val in enumerate(states.most_common()):
    st, freq = val
    print(f'{rank + 1:<3} {st}  {freq:>10,}')

1   ACTIVE   3,355,483
2   CEASED     485,414
3   ON LEAVE      42,401
4   SEASONAL      33,451
5   ON SEPARATION LEAVE       6,541


In [38]:
# empty values were cleaned after data cleaning

profiles = ds.profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
Fiscal Year,3923290,0,7,0.000002,2.805614
Payroll Number,3923290,1745440,157,0.000072,4.286506
Agency Name,3923290,0,165,0.000042,4.365925
Last Name,3923290,2031,157080,0.040059,14.264455
First Name,3923290,2033,88232,0.022501,11.611521
Mid Init,3923290,1596166,43,0.000018,4.073274
Agency Start Date,3923290,63,14933,0.003806,11.097847
Work Location Borough,3923290,0,18,0.000005,1.769786
Title Description,3923290,0,1762,0.000449,6.206462
Leave Status as of June 30,3923290,0,5,0.000001,0.710495


In [7]:
import pandas as pd
from openclean_notebook import DB
db = DB(basedir='.openclean', create=True)

ds = db.load_dataset(source=ds_full, name='ds')
db.edit('ds', n=100)